In [1]:
import re
import requests
import pandas as pd
from env import hud_token

In [3]:
# getting root url
url = 'https://www.huduser.gov/hudapi/public/fmr/listMetroAreas'

header = {'Authorization': f'Bearer {hud_token}'}

response = requests.get(url, headers=header)

data = response.json()

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df[df['area_name'].str.startswith('San Antonio') == True]

In [ ]:
# getting entity_id
entity_id = 'METRO41700M41700'

In [ ]:
url = f'https://www.huduser.gov/hudapi/public/fmr/data/{entity_id}'
response = requests.get(url, headers=header)
data = response.json()

In [ ]:
data['data']['year']

In [ ]:
df = pd.DataFrame(data['data']['basicdata'])

In [ ]:
# adding year
df['year'] = data['data']['year']

In [ ]:
df

In [ ]:
years = ['2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010' ]

for year in years:
    url = f'https://www.huduser.gov/hudapi/public/fmr/data/{entity_id}?year={year}'
    response = requests.get(url, headers=header)
    print(year, response)

# no data available for the metro area

#### Takeaways:
* The data is not avaiable until 2017

#### Actions:
* Make a for loop that creates a dummy df and adds the information from the newly acquired data to it 

#### Later inspection
* What is the reason for this? Was the zipcode data not available? Why is this the case? Will this impact our results?


In [ ]:
years = ['2022', '2021', '2020', '2019', '2018']

for year in years:
    url = f'https://www.huduser.gov/hudapi/public/fmr/data/{entity_id}?year={year}'
    response = requests.get(url, headers=header)
    data = response.json()
    dum = pd.DataFrame(data['data']['basicdata'])
    dum['year'] = data['data']['year']
    df = pd.concat([df, dum])
    
df

##### Takeaways:
* We wonly get data from 2018 and onward due to data challenges with zipcodes not being present
* Once again, this needs to investigated but in the future

In [49]:
# basic requirements
entity_id = 'METRO41700M41700'
header = {'Authorization': f'Bearer {hud_token}'}

# accessing api
url = f'https://www.huduser.gov/hudapi/public/fmr/data/{entity_id}'
response = requests.get(url, headers=header)
data = response.json()

# getting initial df
df = pd.DataFrame()

# establishing years to pull
years = ['2023', '2022', '2021', '2020', '2019', '2018']

# For loop to access data from each year and concat onto the initial df
for year in years:
    url = f'https://www.huduser.gov/hudapi/public/fmr/data/{entity_id}?year={year}'
    response = requests.get(url, headers=header)
    data = response.json()
    dum = pd.DataFrame(data['data']['basicdata'])
    dum['year'] = data['data']['year']
    df = pd.concat([df, dum])
    
df

,zip_code,Efficiency,One-Bedroom,Two-Bedroom,Three-Bedroom,Four-Bedroom,year
0,78002,760,870,1060,1350,1640,2023
1,78003,790,910,1110,1410,1710,2023
2,78006,1030,1240,1530,1870,2340,2023
3,78009,840,890,1170,1530,1570,2023
4,78010,790,910,1100,1400,1710,2023
...,...,...,...,...,...,...,...
130,78676,800,950,1160,1560,1880,2018
131,78850,560,690,870,1150,1380,2018
132,78883,610,760,950,1250,1520,2018
133,78884,560,690,870,1150,1380,2018


In [15]:
zip_codes = df['zip_code'].unique()

# Zillow API

In [26]:
url = "https://zillow56.p.rapidapi.com/search"

querystring = {"location":"78002"}

headers = {
	"X-RapidAPI-Key": "114e895469msh94a2c2aa405c447p161c7ejsn7b09da3c1151",
	"X-RapidAPI-Host": "zillow56.p.rapidapi.com"
}

In [27]:
response = requests.get(url, headers=headers, params=querystring)
data = response.json()

In [30]:
data

{'message': 'You have exceeded the MONTHLY quota for Requests on your current plan, BASIC. Upgrade your plan at https://rapidapi.com/s.mahmoud97/api/zillow56'}

In [24]:
pd.DataFrame(data['results'])

KeyError: 'results'

In [ ]:


for zip_code in zip_codes:
    querystring = {"location":f"{str(zip_code)}"}
    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()
    print(zip_code, data[totalPages])

In [12]:
querystring = {"location":f"{}"}
response = requests.get(url, headers=headers, params=querystring)
data = response.json()

Split Zipcodes

In [47]:
corey = zip_codes[:49]
mack = zip_codes[49:99]
aaron = zip_codes[99:]

In [48]:
aaron

array(['78258', '78259', '78260', '78261', '78263', '78264', '78265',
       '78266', '78268', '78269', '78270', '78278', '78279', '78280',
       '78283', '78285', '78288', '78291', '78292', '78293', '78294',
       '78295', '78296', '78297', '78298', '78299', '78606', '78623',
       '78638', '78648', '78655', '78666', '78670', '78676', '78883',
       '78884', '78885', '78117', '78850'], dtype=object)

In [50]:
rapid_api_key = "114e895469msh94a2c2aa405c447p161c7ejsn7b09da3c1151"

headers = {
	"X-RapidAPI-Key": rapid_api_key,
	"X-RapidAPI-Host": "realty-mole-property-api.p.rapidapi.com"
}
zipcode = '78258'
url = f"https://realty-mole-property-api.p.rapidapi.com/zipCodes/{zipcode}"
response = requests.get(url, headers=headers)
data = response.json()

In [57]:
df = pd.DataFrame()
dum = pd.DataFrame(data)
dum.head()

,id,zipCode,rentalData
averageRent,78258,78258,2761
detailed,78258,78258,"[{'bedrooms': 1, 'averageRent': 1366, 'minRent..."
history,78258,78258,"{'2020-04': {'averageRent': 1788.12, 'minRent'..."
maxRent,78258,78258,8750
minRent,78258,78258,956


In [65]:
# The detailed section pulled out
pd.DataFrame(dum.loc['detailed', 'rentalData'])

,bedrooms,averageRent,minRent,maxRent,totalRentals
0,1,1366,956,1920,13
1,2,1391,1135,1550,3
2,3,2475,1707,4750,62
3,4,2998,2000,6000,83
4,5,3470,2850,5000,17
5,6,6875,5000,8750,2


In [70]:
history = pd.DataFrame(dum.loc['history', 'rentalData']).T

In [79]:
history.loc[:,'detailed']

2020-04    [{'bedrooms': 0, 'averageRent': 937.28, 'minRe...
2020-05    [{'bedrooms': 0, 'averageRent': 973.12, 'minRe...
2020-06    [{'bedrooms': 0, 'averageRent': 980.08, 'minRe...
2020-07    [{'bedrooms': 0, 'averageRent': 982.12, 'minRe...
2020-08    [{'bedrooms': 0, 'averageRent': 987.68, 'minRe...
2020-09    [{'bedrooms': 0, 'averageRent': 1004.74, 'minR...
2020-10    [{'bedrooms': 0, 'averageRent': 1004.3, 'minRe...
2020-11    [{'bedrooms': 0, 'averageRent': 1007.75, 'minR...
2020-12    [{'bedrooms': 0, 'averageRent': 996.48, 'minRe...
2021-01    [{'bedrooms': 0, 'averageRent': 996.48, 'minRe...
2021-02    [{'bedrooms': 0, 'averageRent': 1002.32, 'minR...
2021-03    [{'bedrooms': 0, 'averageRent': 1002.32, 'minR...
2021-04    [{'bedrooms': 0, 'averageRent': 1002.32, 'minR...
2021-05    [{'bedrooms': 0, 'averageRent': 1012.84, 'minR...
2021-06    [{'bedrooms': 0, 'averageRent': 1012.84, 'minR...
2021-11    [{'bedrooms': 0, 'averageRent': 2113.16, 'minR...
2021-12    [{'bedrooms':

In [ ]:
df = pd.DataFrame()

for zipcode in aaron:
    url = f"https://realty-mole-property-api.p.rapidapi.com/zipCodes/{zipcode}"
    response = requests.get(url, headers=headers)
    data = response.json()
    dum = pd.DataFrame(data)
    df = pd.concat([df, dum])

# response = requests.get(url, headers=headers)

data = response.json()

Completed Query

In [ ]:
# basic requirements
entity_id = 'METRO41700M41700'
headers = {'Authorization': f'Bearer {hud_token}'}

# accessing api
url = f'https://www.huduser.gov/hudapi/public/fmr/data/{entity_id}'
response = requests.get(url, headers=header)
data = response.json()

# getting initial df
df = pd.DataFrame()

# establishing years to pull
years = ['2023', '2022', '2021', '2020', '2019', '2018']

# For loop to access data from each year and concat onto the initial df
for year in years:
    url = f'https://www.huduser.gov/hudapi/public/fmr/data/{entity_id}?year={year}'
    response = requests.get(url, headers=header)
    data = response.json()
    dum = pd.DataFrame(data['data']['basicdata'])
    dum['year'] = data['data']['year']
    df = pd.concat([df, dum])
    
df

https://rapidapi.com/realtymole/api/realty-mole-property-api